In [1]:
# Load common imports
%run ./CommonImports.ipynb

In [2]:
# Loss function 1
def calculate_loss_function_around_one(weights):
    weights = weights / weights.sum()
    amounts = weights * reference_date_exhange_rate

    # TODO: penalize large coefficients?
    loss_function_score = 0

    for base_currency in currencies:
        amounts_through_time = amounts * exchange_table[base_currency]
        normalized_amounts_through_time = amounts_through_time.sum(axis=1) / reference_date_exhange_rate[base_currency]
        
        # TODO: other possibilities for loss functions:
        # - Mean Absolute Error MAE
        # - Root Mean Squared Error RMSE
        # - Time-series related loss functions (research)
        
        deviation = np.log(normalized_amounts_through_time)
        loss_function_score += (deviation**2).mean()
    
    return loss_function_score

In [3]:
# Loss function 2
def calculate_loss_function_vs_t_minus_one(weights):
    weights = weights / weights.sum()
    amounts = weights * reference_date_exhange_rate.loc[p13_currencies]

    loss_function_score = 0

    for base_currency in p13_currencies:
        amounts_through_time = (amounts * exchange_table[base_currency]).sum(axis=1)
        loss_function_score += (np.abs(amounts_through_time - amounts_through_time.shift(1))/amounts_through_time).mean()

    return loss_function_score

In [4]:
# Loss function 3
def calculate_loss_function_vs_t_minus_one_with_balancing(weights, currencies):
#     print("evaluate")
    
    # Ensure we never deviate from 100% total, and only use positive numbers
    weights = np.abs(weights) / np.abs(weights).sum()  
    
    # Calculate the weights after rebalancing for each month
    amounts_by_month = {month: weights / this_day_by_month["EUR"][month].iloc[0][currencies] for month in months}
    
    # Calculate the loss function per base currency, per month
    return sum([sum([(amounts_by_month[month] * diff_by_month[base_currency][month]).sum(axis=1).sum() for month in months]) 
                for base_currency in currencies])

In [5]:
# Loss function 4
def calculate_loss_function_vs_t_minus_one_for_period(weights, max_date):
    weights = weights / weights.sum()
    amounts = weights * reference_date_exhange_rate.loc[p13_currencies]

    loss_function_score= 0

    for base_currency in p13_currencies:
        amounts_through_time = (amounts * exchange_table[base_currency].loc[:max_date]).sum(axis=1)
        loss_function_score += (np.abs(amounts_through_time - amounts_through_time.shift(1))/amounts_through_time).mean()

    return loss_function_score